In [1]:
#importing packages & changing settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 200)

In [2]:
# reading datasets
pc = pd.read_csv('postcode_data.csv')
# vol = pd.read_csv('afmetingen2.csv') # dataset of past few years FIRST DATASET
vol = pd.read_csv('a7d3ad0e-8fa2-4295-88bb-ac6aab185d02.csv') # dataset of past few years SECOND DATASET


# merging the datasets on bl_code
df= pc.merge(vol, on= 'bl_code')

# drop columns that will not be used
# df.drop(['afmeting', 'aantal', 'bl_code'], axis=1, inplace=True)

In [3]:
# convert to the correct datatype
df['datum']=pd.to_datetime(df['datum'])
df= df[(df['datum'] >= '2020-01-01')]
df.sort_values(by='datum', inplace=True)

In [4]:
# check datatypes & missing values
print (df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7031586 entries, 4214003 to 3911983
Data columns (total 6 columns):
 #   Column    Dtype         
---  ------    -----         
 0   bl_code   int64         
 1   postcode  object        
 2   datum     datetime64[ns]
 3   afmeting  object        
 4   volume    float64       
 5   aantal    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 375.5+ MB
None


In [5]:
# create a postalcode-3 level based on postalcode-6
post= []
for i in df['postcode']:
    post.append( i[:3])
    
df['pc3']= post

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7031586 entries, 4214003 to 3911983
Data columns (total 7 columns):
 #   Column    Dtype         
---  ------    -----         
 0   bl_code   int64         
 1   postcode  object        
 2   datum     datetime64[ns]
 3   afmeting  object        
 4   volume    float64       
 5   aantal    int64         
 6   pc3       object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 429.2+ MB


### removing outliers

In [7]:
# checking the outliers

print('index of outliers',np.where((df['volume']> 300) ))
outliers = np.where((df['volume']> 300) )
print()
out= df.iloc[outliers]
out

index of outliers (array([4547314, 4548550, 4571944, 4572569, 6260887, 6266826, 6469023]),)



,bl_code,postcode,datum,afmeting,volume,aantal,pc3
6128958,36267,3541CH,2021-11-17,groot,9997.017,2,354
2911015,37029,1025ET,2021-11-17,groot,9997.000,1,102
6039675,5569,7411HJ,2021-11-20,groot,9997.000,1,741
2765469,35723,3811LK,2021-11-20,groot,19994.001,2,381
3843524,3429,2031AZ,2022-07-18,groot,502.224,72,203
2412145,15493,2222AH,2022-07-19,groot,375.000,1,222
3843700,3429,2031AZ,2022-08-17,groot,336.459,38,203


### cleaning postal code

In [8]:
# cbs dataset and change to string from int
cbs =pd.read_excel('cbs_pc4_2020_v1.xlsx' )
cbs['PC4'] = pd.Series(cbs['PC4'], dtype="string")

In [9]:
# add column to the cbs dataset with pc3 from pc4
p3= []
for i in cbs['PC4']:
    p3.append( i[:3])
    
cbs['p3']= p3    
print(len(p3))
print(p3[:5])

4068
['101', '101', '101', '101', '101']


In [10]:
print('PostNL dataset :', df['pc3'].nunique())
print('CBS dataset :', cbs['p3'].nunique())

PostNL dataset : 841
CBS dataset : 798


In [11]:
be = [230, 300, 350, 380, 897, 890, 970, 339, 288, 359, 879, 800, 898, 309, 366, 120, 330, 367, 200, 878, 100, 368, 820, 869, 919, 180, 149, 500, 460, 750, 730, 480, 620, 369, 497, 137, 250, 496, 866, 717 ,780, 680, 650, 560, 260, 519, 188, 678, 918, 895, 868, 867, 718, 719, 157]
belgie = [str(x) for x in be]

In [12]:
df = df[~df.pc3.isin(belgie)]

In [13]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6983505 entries, 4214003 to 3911983
Data columns (total 7 columns):
 #   Column    Dtype         
---  ------    -----         
 0   bl_code   int64         
 1   postcode  object        
 2   datum     datetime64[ns]
 3   afmeting  object        
 4   volume    float64       
 5   aantal    int64         
 6   pc3       object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 426.2+ MB


In [14]:
df=df.drop(['postcode'], axis=1)

In [15]:
df['pc3'].nunique()

786

### create new dataset with total volume (m3) per pc3 per day

In [16]:
# create new df where the total vol per postalcode-3 per day is in one row  ONLY SUM 

new = df.groupby(['datum', 'pc3']).agg({'volume': ['sum']})
new.columns = ['volume_per_pc3']
new = new.reset_index()

### adding pc3 rows on days that there was no volume

In [17]:
new.head()

,datum,pc3,volume_per_pc3
0,2020-01-01,101,0.155
1,2020-01-01,102,0.192
2,2020-01-01,105,0.113
3,2020-01-01,106,0.156
4,2020-01-01,107,1.454


In [18]:
# making all pc areas a column and if value is missing per day it is filled with 0. 
pivot = new.pivot_table(index='datum',columns='pc3',values='volume_per_pc3',aggfunc='sum',  fill_value=0).reset_index()

In [19]:
# list of all pc3 areas
pcs = list(pivot.columns)
pcs=pcs[1:]

In [20]:
pivot

pc3,datum,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,121,122,123,124,125,...,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,2020-01-01,0.155,0.192,0.000,0.000,0.113,0.156,1.454,0.507,0.132,0.034,0.253,0.000,0.109,0.000,0.000,0.000,0.068,0.161,0.000,0.000,1.084,0.000,0.000,0.000,...,0.059,0.000,0.000,0.097,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.138,0.000,0.000,0.000,0.000,0.000,0.000
1,2020-01-02,26.745,6.187,8.805,0.011,27.328,25.487,27.859,9.122,16.908,9.523,12.286,2.158,5.152,3.098,0.544,1.952,3.901,18.767,2.280,35.850,3.770,2.802,1.815,2.873,...,4.531,3.730,0.317,1.296,0.853,3.279,0.0,0.271,0.228,0.855,0.000,3.050,0.028,1.031,0.033,3.364,0.595,0.000,2.059,0.353,2.054,1.151,0.335,2.584,0.564
2,2020-01-03,32.760,5.167,8.947,0.270,25.909,23.523,22.500,9.851,16.517,9.205,11.265,1.946,4.831,1.616,0.458,2.344,2.632,17.957,1.420,15.900,3.748,3.027,2.001,2.146,...,4.713,2.038,0.244,1.233,1.741,2.942,0.0,0.235,0.180,0.708,0.000,2.417,0.009,0.295,0.046,2.761,0.952,0.000,2.016,1.105,2.953,0.800,0.261,8.013,1.777
3,2020-01-04,21.934,5.913,8.310,0.000,27.818,20.777,28.654,10.377,17.024,9.478,9.600,2.049,4.782,2.199,0.739,2.068,3.086,12.831,2.217,10.807,3.947,1.440,1.844,2.598,...,3.634,2.272,0.124,1.106,1.120,2.178,0.0,0.224,0.250,1.148,0.000,1.347,0.000,0.325,0.027,2.050,0.739,0.001,1.773,0.322,1.483,0.751,0.148,2.123,0.519
4,2020-01-05,5.924,1.191,2.827,0.000,7.220,3.369,8.055,6.085,1.986,0.416,1.884,0.000,1.179,0.000,0.000,0.000,0.294,3.647,0.000,2.209,2.509,0.922,0.749,1.557,...,1.115,0.000,0.000,0.102,0.000,0.399,0.0,0.000,0.000,0.000,0.000,0.130,0.000,0.000,0.000,0.969,0.000,0.000,0.318,0.092,0.692,0.150,0.064,0.034,0.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036,2022-11-02,35.118,5.881,12.307,0.717,34.558,36.899,26.775,10.424,18.627,15.297,12.721,4.292,7.965,3.085,0.244,2.223,7.802,16.211,2.598,18.925,3.600,4.407,2.277,2.697,...,5.597,3.640,0.177,2.560,0.754,3.557,0.0,0.825,0.432,1.056,0.305,2.219,0.056,0.021,1.079,4.351,1.111,0.000,4.798,0.745,2.814,1.305,0.342,4.680,0.744
1037,2022-11-03,35.624,5.477,11.717,0.520,28.059,44.882,24.610,9.573,17.211,16.336,12.118,3.276,7.935,1.417,0.448,3.327,6.894,15.537,1.581,17.453,4.030,5.033,2.279,2.381,...,3.995,4.843,0.198,2.039,0.876,5.684,0.0,0.994,0.225,1.275,0.016,2.666,0.025,0.007,0.292,4.650,1.327,0.000,3.039,1.078,3.662,1.293,0.451,3.987,1.345
1038,2022-11-04,30.183,4.692,11.475,0.060,28.756,35.210,26.903,10.216,13.457,17.171,9.967,3.301,9.793,2.228,1.126,2.391,5.157,16.977,2.248,16.617,5.354,4.148,3.048,2.333,...,4.898,3.901,0.335,2.277,1.072,5.496,0.0,1.485,0.140,1.163,0.023,2.555,0.000,0.006,0.109,3.924,0.709,0.000,3.363,0.906,3.227,1.780,0.586,4.908,0.747
1039,2022-11-05,11.626,1.976,3.721,0.000,15.303,11.388,11.957,6.977,5.237,5.998,3.360,1.016,2.261,1.072,0.000,0.515,1.116,8.145,0.703,10.568,3.262,1.376,1.914,1.362,...,2.934,2.680,0.000,2.796,0.616,2.537,0.0,0.000,0.000,0.801,0.000,0.681,0.000,0.000,0.000,1.298,0.709,0.000,1.479,0.000,2.280,1.047,0.181,2.074,0.354


In [21]:
lala= pivot.melt( id_vars =['datum'], value_vars =pcs, value_name= 'volume_per_pc3')

In [22]:
lala= lala.sort_values(['pc3','datum'])
lala.head()

,datum,pc3,volume_per_pc3
0,2020-01-01,101,0.155
1,2020-01-02,101,26.745
2,2020-01-03,101,32.760
3,2020-01-04,101,21.934
4,2020-01-05,101,5.924


### Add weekday column 

In [23]:
lala['weekday'] = lala['datum'].dt.day_name()

In [24]:
lala.loc[(lala['pc3'] == '609')& (lala['weekday'] == 'Sunday')]

,datum,pc3,volume_per_pc3,weekday
463249,2020-01-05,609,0.123,Sunday
463256,2020-01-12,609,0.301,Sunday
463263,2020-01-19,609,0.388,Sunday
463270,2020-01-26,609,0.123,Sunday
463277,2020-02-02,609,0.324,Sunday
463284,2020-02-09,609,0.291,Sunday
463291,2020-02-16,609,0.293,Sunday
463298,2020-02-23,609,0.007,Sunday
463305,2020-03-01,609,0.246,Sunday
463312,2020-03-08,609,0.087,Sunday


In [25]:
# manually check if it was done correctly
# df.loc[(df['pc3'] == '104')& (df['datum'] == '2022-08-15')] 

In [26]:
print('PostNL dataset :', lala['pc3'].nunique())

PostNL dataset : 786


### Add lockdown feature

In [27]:
conditions = [
    (lala['datum'] >= '2020-12-14' ) & (lala['datum'] <= '2021-02-07'),
    (lala['datum'] >= '2021-02-08') & (lala['datum'] <= '2021-04-28')]
choices = [1, 2]
lala['lockdown'] = np.select(conditions, choices, default=0)

In [28]:
# lala.loc[lala['datum'] == '2020-12-14']

In [29]:
# lala.loc[lala['datum'] == '2021-04-28']

In [30]:
# lala.loc[lala['datum'] == '2021-04-29']

In [31]:
lala.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 818226 entries, 0 to 818225
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   datum           818226 non-null  datetime64[ns]
 1   pc3             818226 non-null  object        
 2   volume_per_pc3  818226 non-null  float64       
 3   weekday         818226 non-null  object        
 4   lockdown        818226 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 37.5+ MB


### Saving new file to csv

In [32]:
lala.to_csv('newdata.csv', index=False)

In [33]:
test= pd.read_csv('newdata.csv')
test.head()

,datum,pc3,volume_per_pc3,weekday,lockdown
0,2020-01-01,101,0.155,Wednesday,0
1,2020-01-02,101,26.745,Thursday,0
2,2020-01-03,101,32.760,Friday,0
3,2020-01-04,101,21.934,Saturday,0
4,2020-01-05,101,5.924,Sunday,0


In [ ]:
test.info()

### Dataset with only PC3 level 101 for the whole time period

In [ ]:
# new dataset with only data of pc3 '101'
pc101 = lala[lala.pc3 == '101']

In [ ]:
pc101.head(10)

In [ ]:
#check of nieuwe dataset maken goed gegaan is
for i in pc101['pc3']:
    if i  != '101':
        print (i)

In [ ]:
pc101.plot(y='volume_per_pc3', x='datum', figsize=(20,7))

#### Creating a dataset for every postal code